# This notebook is to prototype vehicle make prediction

## Establish connection with EVA server

In [1]:
import sys
sys.path.insert(0,'..')
from src.server.db_api import connect

In [2]:
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

In [3]:
cursor = connection.cursor()

### Upload the survellience video to analyse

In [4]:
cursor.execute('UPLOAD INFILE "../data/bdd_test/bdd_1.mp4" PATH "video.mp4";')
response = cursor.fetch_all()
print(response)

Response Object:
@status: 0
@batch: Batch Object:
@dataframe: Empty DataFrame
Columns: []
Index: []
@batch_size: 0
@identifier_column: id
@metrics: None


## Visualize video

In [5]:
from ipywidgets import Video
Video.from_file("../data/bdd_test/bdd_1.mp4", embed=True)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00<wmoov\x00\x00\x00lmvhd\x00\x00\x00…

## Load video into EVA

In [6]:
cursor.execute('LOAD DATA INFILE "video.mp4" INTO MyVideo;')
response = cursor.fetch_all()
print(response)

Response Object:
@status: 0
@batch: Batch Object:
@dataframe:        Video  Num Loaded Frames
0  video.mp4               1203
@batch_size: 1
@identifier_column: id
@metrics: None


## Definte a new UDF called VehicleMakePredictor if it does not already exist

In [ ]:
cursor.execute("""CREATE UDF IF NOT EXISTS VehicleMakePredictor
      INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
      OUTPUT (labels NDARRAY STR(ANYDIM), bboxes NDARRAY FLOAT32(ANYDIM, 4),
                scores NDARRAY FLOAT32(ANYDIM))
      TYPE  Classification
      IMPL  'src/udfs/vehicle_make_predictor.py';
      """)
response = cursor.fetch_all()
print(response)

## Run vehicle predictor on video

In [7]:
cursor.execute("""SELECT id, Unnest(VehicleMakePredictor(data)) FROM MyVideo""")
response = cursor.fetch_all()

In [8]:
response.batch.frames

,id,boxes,labels,makes,scores
0,1067,"[[792.8865966797, 431.6767883301], [872.873107...",car,[[Geo Metro Convertible 1993]],0.998317
1,1067,"[[554.9195556641, 396.9907531738], [772.661926...",car,[[Daewoo Nubira Wagon 2002]],0.996497
2,1067,"[[166.4351348877, 447.8352661133], [236.469070...",car,[[Geo Metro Convertible 1993]],0.942952
3,1067,"[[109.8822021484, 445.976348877], [170.6081848...",car,[[GMC Acadia SUV 2012]],0.903357
4,1067,"[[770.5823364258, 420.3346252441], [798.956298...",car,[[Ram C/V Cargo Van Minivan 2012]],0.896569
...,...,...,...,...,...
5033,785,"[[536.6068725586, 423.2253112793], [620.289123...",car,[[Geo Metro Convertible 1993]],0.997436
5034,785,"[[224.9058380127, 378.519317627], [548.0108032...",car,[[Suzuki Aerio Sedan 2007]],0.997001
5035,785,"[[1023.4869995117, 413.2431640625], [1070.1868...",car,[[GMC Savana Van 2012]],0.978854
5036,785,"[[626.7808837891, 433.8468322754], [651.508911...",car,[[Geo Metro Convertible 1993]],0.972461


### Visualize output of vehicle make detector on the video

In [9]:
import cv2
def annotate_video(detections, input_video_path, output_video_path):
    color=(0,255,0)
    thickness=3

    vcap = cv2.VideoCapture(input_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc(*'H264') #codec
    video=cv2.VideoWriter(output_path, fourcc, fps, (width,height))

    frame_id = 0
    # Capture frame-by-frame
    ret, frame = vcap.read()  # ret = 1 if the video is captured; frame is the image

    while ret:
        df = detections
        df = df[['boxes', 'labels', 'makes']][df.id == frame_id]
        if df.size:
            for bbox, label, make in df.values:
                # Though the model is trained on the actual car version itself, lets just print out the maker for now
                maker = make[0][0].split(" ")[0]
                x1, y1 = bbox[0]
                x2, y2 = bbox[1]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                img=cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness) 
                cv2.putText(img, maker, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness-1) # object label
            video.write(img)

        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [11]:
from ipywidgets import Video
input_path = '../data/bdd_test/bdd_1.mp4'
output_path = 'video.mp4'
annotate_video(response.batch.frames, input_path, output_path)
Video.from_file(output_path)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x01\x14QOmdat\x00\x00\…